In [16]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F
from torch.autograd import Variable
import numpy as np
from matplotlib import pyplot as plt
import torchvision.datasets as dsets
import torchvision.transforms as transforms

# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.01

# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


In [26]:
class LogisticRegression(nn.Module):
    def __init__(self, input_s, output_s):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_s, output_s)
    
    def sigmoid(self, x):
        return (1 / (1+torch.exp(x)))
    
    def forward(self, x):
        logit = self.linear(x)
        #out = self.sigmoid(logit)
        return logit

In [27]:
LR = LogisticRegression(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(LR.parameters(), lr=learning_rate)

In [28]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
    
        optimizer.zero_grad()
        output = LR(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
                   % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))    

Epoch: [1/5], Step: [100/600], Loss: 1.5641
Epoch: [1/5], Step: [200/600], Loss: 1.2090
Epoch: [1/5], Step: [300/600], Loss: 1.0395
Epoch: [1/5], Step: [400/600], Loss: 0.8930
Epoch: [1/5], Step: [500/600], Loss: 0.6872
Epoch: [1/5], Step: [600/600], Loss: 0.7793
Epoch: [2/5], Step: [100/600], Loss: 0.7238
Epoch: [2/5], Step: [200/600], Loss: 0.5971
Epoch: [2/5], Step: [300/600], Loss: 0.6241
Epoch: [2/5], Step: [400/600], Loss: 0.6041
Epoch: [2/5], Step: [500/600], Loss: 0.6012
Epoch: [2/5], Step: [600/600], Loss: 0.5566
Epoch: [3/5], Step: [100/600], Loss: 0.6863
Epoch: [3/5], Step: [200/600], Loss: 0.6306
Epoch: [3/5], Step: [300/600], Loss: 0.4170
Epoch: [3/5], Step: [400/600], Loss: 0.5239
Epoch: [3/5], Step: [500/600], Loss: 0.5201
Epoch: [3/5], Step: [600/600], Loss: 0.4426
Epoch: [4/5], Step: [100/600], Loss: 0.4873
Epoch: [4/5], Step: [200/600], Loss: 0.6391
Epoch: [4/5], Step: [300/600], Loss: 0.4680
Epoch: [4/5], Step: [400/600], Loss: 0.4568
Epoch: [4/5], Step: [500/600], L

In [33]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = LR(images)  
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

# Save the Model
torch.save(LR.state_dict(), 'logistic_regression.pkl')

Accuracy of the model on the 10000 test images: 89 %
